In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('simulationDataClean .csv')
df

,date,event,fighter1,fighter1 - open,fighter1 - close,fighter2,fighter2 - open,fighter2 - close,winner
0,2025-03-09,UFC 313,Francis Marshall,"Mon, Feb 24, 2025 18:15 - price: +290","Sat, Mar 8, 2025 19:01 - price: +197",Mairon Santos,"Mon, Feb 24, 2025 18:15 - price: -370","Sat, Mar 8, 2025 19:01 - price: -245",Mairon Santos
1,2025-03-09,UFC 313,King Green,"Mon, Feb 24, 2025 17:55 - price: +430","Sat, Mar 8, 2025 21:01 - price: +398",Mauricio Ruffy,"Mon, Feb 24, 2025 17:55 - price: -600","Sat, Mar 8, 2025 21:01 - price: -540",Mauricio Ruffy
2,2025-03-09,UFC 313,Rafael Fiziev,"Thu, Feb 27, 2025 05:11 - price: -139","Sat, Mar 8, 2025 23:01 - price: -159",Justin Gaethje,"Thu, Feb 27, 2025 05:11 - price: +112","Sat, Mar 8, 2025 23:01 - price: +131",Justin Gaethje
3,2025-03-09,UFC 313,Brunno Ferreira,"Mon, Feb 24, 2025 18:07 - price: +105","Sat, Mar 8, 2025 21:01 - price: +112",Armen Petrosyan,"Mon, Feb 24, 2025 18:07 - price: -125","Sat, Mar 8, 2025 21:01 - price: -134",Brunno Ferreira
4,2025-03-09,UFC 313,Magomed Ankalaev,"Sat, Jan 18, 2025 23:01 - price: +142","Sat, Mar 8, 2025 23:01 - price: +104",Alex Pereira,"Sat, Jan 18, 2025 23:01 - price: -170","Sat, Mar 8, 2025 23:01 - price: -125",Magomed Ankalaev
...,...,...,...,...,...,...,...,...,...
5192,2007-07-07,UFC 73: Stacked,Nate Marquardt,"Mon, Jul 2, 2007 01:00 - price: +150","Sat, Jul 7, 2007 01:00 - price: +125",Anderson Silva,"Mon, Jul 2, 2007 01:00 - price: -170","Sat, Jul 7, 2007 01:00 - price: -145",Anderson Silva
5193,2007-07-07,UFC 73: Stacked,Heath Herring,"Tue, Jul 3, 2007 00:21 - price: +470","Sat, Jul 7, 2007 18:30 - price: +500",Antonio Rodrigo Nogueira,"Tue, Jul 3, 2007 00:21 - price: -810","Sat, Jul 7, 2007 18:30 - price: -700",Antonio Rodrigo Nogueira
5194,2007-07-07,UFC 73: Stacked,Mark Bocek,"Mon, Jul 2, 2007 18:00 - price: +180","Sun, Jul 8, 2007 00:30 - price: +190",Frankie Edgar,"Mon, Jul 2, 2007 18:00 - price: -260","Sun, Jul 8, 2007 00:30 - price: -230",Frankie Edgar
5195,2007-07-07,UFC 73: Stacked,Hermes Franca,"Mon, Jul 2, 2007 01:00 - price: +200","Sun, Jul 8, 2007 00:30 - price: +225",Sean Sherk,"Mon, Jul 2, 2007 01:00 - price: -240","Sun, Jul 8, 2007 00:30 - price: -265",Sean Sherk


In [3]:
print(df.isna().any(axis=1).sum())
df = df.dropna().reset_index(drop=True)
print(df.isna().any(axis=1).sum())

94
0


In [4]:
def calculate_profit(wager, price):
    if price > 0:
        profit = (price/100)*wager
    else:
        profit = (100/abs(price))*wager
    return profit


In [5]:
wager = 100
bankroll = wager * len(df)

profit_dict = {
    'favorite_at_open': bankroll,
    'favorite_at_close': bankroll,

    'underdog_at_open': bankroll,
    'underdog_at_close': bankroll,

    'favorite_random': bankroll,
    'underdog_random': bankroll,
    
    'true_random': bankroll
}
print(profit_dict)

{'favorite_at_open': 510300, 'favorite_at_close': 510300, 'underdog_at_open': 510300, 'underdog_at_close': 510300, 'favorite_random': 510300, 'underdog_random': 510300, 'true_random': 510300}


In [6]:
import random

for index, row in df.iterrows():
    fighter1 = row['fighter1']
    fighter2 = row['fighter2']
    winner = row['winner']

    #get open line
    open1 = int(row['fighter1 - open'].split('price: ')[1])
    open2 = int(row['fighter2 - open'].split('price: ')[1])

    close1 = int(row['fighter1 - close'].split('price: ')[1])
    close2 = int(row['fighter2 - close'].split('price: ')[1])

    lines = {
        fighter1: [open1, close1],
        fighter2: [open2, close2]
    }

    for_random = [open1, open2, close1, close2]

    neg_lines = []
    pos_lines = []
    for line in for_random:
        if(line > 0):
            pos_lines.append(line)
        else:
            neg_lines.append(line)

    if(len(pos_lines) >0):
        #dog random blindly takes a line
        random_dog = random.choice(pos_lines)
    else:
        #no pos line default to closest to pos
        random_dog = max(for_random)

    if(len(neg_lines)> 0):
        #fav random blindly takes a line
        random_fav = random.choice(neg_lines)
    else:
        random_fav = min(for_random)

    line_list = lines[winner]

    #if random fav picked a winning line
    if random_fav in line_list:
        profit_dict['favorite_random']+=calculate_profit(wager, random_fav)
    else:
        profit_dict['favorite_random']-=wager

    #if random dog picked a winning line
    if random_dog in line_list:
        profit_dict['underdog_random']+=calculate_profit(wager, random_dog)
    else:
        profit_dict['underdog_random']-=wager



    #true random
    random_fighter = row[f'fighter{random.randint(1,2)}']
    random_line = random.randint(0,1)
    true_random = lines[random_fighter][random_line]
    

    #if true random picked a winning line
    if true_random in line_list:
        profit_dict['true_random']+=calculate_profit(wager, true_random)
    else:
        profit_dict['true_random']-=wager

    #open fav line wins
    if(line_list[0] < 0):
        #favorite open bettors win
        profit_dict['favorite_at_open']+=calculate_profit(wager, line_list[0])
        #dog open bettor loses
        profit_dict['underdog_at_open']-=wager
    #open fav line loses
    else:
        profit_dict['favorite_at_open']-=wager
        profit_dict['underdog_at_open']+=calculate_profit(wager, line_list[0])

    if(line_list[1] < 0):
        #favorite open bettors win
        profit_dict['favorite_at_close']+=calculate_profit(wager, line_list[1])
        #dog open bettor loses
        profit_dict['underdog_at_close']-=wager
    #open fav line loses
    else:
        profit_dict['favorite_at_close']-=wager
        profit_dict['underdog_at_close']+=calculate_profit(wager, line_list[1])



    
print(profit_dict)




{'favorite_at_open': 538700.7630383957, 'favorite_at_close': 525269.5907914366, 'underdog_at_open': 407019.0, 'underdog_at_close': 440019.0, 'favorite_random': 504645.9538667111, 'underdog_random': 466372.1887817592, 'true_random': 493372.8796426199}
